In [1]:
# # Import dependencies
# import pandas as pd
# from pathlib import Path
# from collections import Counter
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import balanced_accuracy_score, confusion_matrix
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import StandardScaler


In [4]:
# # Create filepath to dataset
# path = "S3bucket"
# EV_stations_df = pd.read_csv('enter_path')

In [5]:
# # Define the target variable
# y = EV_stations_df["Y"]
# X = EV_stations_df.drop(columns="Y")

In [ ]:
# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                    random_state=1,
#                                                    stratify=y)
# X_train.shape

In [7]:
# # Define the model
# model = LinearRegression()

# # Fit the model
# model.fit(X_train, y_train)

# # Make predictions
# y_pred = model.predict(X_test)
# print(y_pred.shape)

In [ ]:
# # print model accuracy
# print(model.coef_)
# print(model.intercept_)